### Usage

In [ ]:
import projspec
proj = projspec.Project("/Users/mdurant/code/projspec", walk=True)

In [ ]:
proj

In [ ]:
"uv" in proj

In [ ]:
proj.all_artifacts()

In [ ]:
proj.all_artifacts("wheel")

In [ ]:
proj.all_artifacts("wheel")[0].to_dict(False)

In [ ]:
proj.all_artifacts("wheel")[1].make()

### Dev

In [ ]:
from projspec.proj.base import ProjectSpec, ProjectExtra
from projspec.proj import PythonLibrary
from projspec.content import BaseContent
from projspec.artifact import BaseArtifact
from projspec.utils import PickleableTomlDecoder
import toml

In [ ]:
# projspec.spec.rust

class Rust(ProjectSpec):
    """A directory, which can build a binary executable or library with Cargo."""

    spec_doc = "https://doc.rust-lang.org/cargo/reference/manifest.html"

    def match(self) -> bool:
        return "Cargo.toml" in self.proj.basenames

    def parse(self):
        pass


class RustPython(Rust, PythonLibrary):
    """A maturin project designed for importing with python, perhaps with mixed rust/python code trees.
    """

    spec_doc = "https://www.maturin.rs/config.html"

    def match(self) -> bool:
        # You can also have metadata.maturin in the Cargo.toml
        return (
            Rust.match(self)
            and ("maturin" in self.proj.pyproject.get("tool", {})
            or self.proj.pyproject.get("build-backend", "") == "maturin")
        )


In [ ]:
example = "/Users/mdurant/code/cramjam//"
proj = projspec.Project(example, walk=True)
proj

In [ ]:
proj = projspec.Project(example, types=["rust", "rust_python"])
proj

Should include:
- artifacts
    - .so, .a, .dll library (cargo)
    - executable (cargo, optional)
    - wheel & source dist (maturin)
- usual python stuff
- rust dependnency environment
- cargo metadata/options

In [ ]:
class Rust(ProjectSpec):
    """A directory, which can build a binary executable or library with Cargo."""

    spec_doc = "https://doc.rust-lang.org/cargo/reference/manifest.html"

    def match(self) -> bool:
        return "Cargo.toml" in self.proj.basenames

    # this builds a (static) library or an executable, or both.
    def parse(self):
        from projspec.content.metadata import DescriptiveMetadata
        with self.proj.fs.open(f"{self.proj.url}/Cargo.toml", "rt") as f:
            meta = toml.load(f, decoder=PickleableTomlDecoder())
        self.contents["desciptive_metadata"] = DescriptiveMetadata(proj=self.proj, meta=meta["package"],
                                                                  artifacts=set())


In [ ]:
print(projspec.Project(example, types=["rust", "rust_python"]))

In [ ]:
class DynamicLibrary(projspec.artifact.FileArtifact):
    ...

from dataclasses import dataclass, field
from projspec.utils import Enum
from enum import auto
# projspec.content.environment
class Stack(Enum):
    """The type of environment by packaging tech"""

    PIP = auto()
    CONDA = auto()
    NPM = auto()
    CARGO = auto()  # new

In [ ]:
def dictify(x):
    return {k: dict(v) if isinstance(v, dict) else v for k, v in x.items()}

class Rust(ProjectSpec):
    """A directory, which can build a binary executable or library with Cargo."""

    spec_doc = "https://doc.rust-lang.org/cargo/reference/manifest.html"

    def match(self) -> bool:
        return "Cargo.toml" in self.proj.basenames

    # this builds a (static) library or an executable, or both.
    def parse(self):
        from projspec.content.metadata import DescriptiveMetadata
        from projspec.content.environment import Environment, Precision
        with self.proj.fs.open(f"{self.proj.url}/Cargo.toml", "rt") as f:
            meta = toml.load(f)
        self.contents["desciptive_metadata"] = DescriptiveMetadata(proj=self.proj, meta=meta["package"],
                                                                  artifacts=set())
        self.contents["environment"] = {
            "default": Environment(proj=self.proj, packages=dictify(meta["dependencies"]), artifacts=set(),
                                  stack=Stack.CARGO, precision=Precision.SPEC),
            "dev": Environment(proj=self.proj, packages=dictify(meta["build-dependencies"]), artifacts=set(),
                              stack=Stack.CARGO, precision=Precision.SPEC)
        }
        if meta.get("lib"):
            ext = ".dylib" # depends on lib type and platform, this is for simplicity
            profile = "debug"
            cmd = ["cargo", "build"]
            # TODO: one for each mata[profile.*], usually debug and release
            self.artifacts["dynamic_library"] = DynamicLibrary(
                proj=self.proj,
                cmd=["cargo", "build", f"--{profile}"],
                fn=f'{self.proj.url}/target/{profile}/{meta["lib"]["name"]}-{meta["package"]["version"]}.{ext}'
            )
        if meta.get("bin"):
            # exeuctable binary artifact
            pass
        # example executable and libraries
        # executable tests

In [ ]:
print(projspec.Project(example, types=["rust", "rust_python"]))

In [ ]:
from projspec.artifact.installable import Wheel

class RustPython(Rust, PythonLibrary):
    """A maturin project designed for importing with python, perhaps with mixed rust/python code trees.
    """

    spec_doc = "https://www.maturin.rs/config.html"

    def match(self) -> bool:
        # You can also have metadata.maturin in the Cargo.toml
        return (
            Rust.match(self)
            and ("maturin" in self.proj.pyproject.get("tool", {})
            or self.proj.pyproject.get("build-backend", "") == "maturin")
        )

    def parse(self):
        PythonLibrary.parse(self)
        # debug and release
        self.artifacts["wheel"] = Wheel(proj=self.proj, cmd=["maturin", "build"],
                                       fn=f"{self.proj.url}/target/wheels/*.whl")
        

In [ ]:
proj = projspec.Project(example, types=["rust", "rust_python"])
print(proj)

In [ ]:
wheel = proj.all_artifacts("wheel")[0]
wheel.clean()
wheel

In [ ]:
wheel.make()
wheel